In [258]:
import numpy as np
import pandas as pd
import spacy
import tensorflow as tf

from random import choice, randint
from sklearn.model_selection import train_test_split
from tensorflow import keras

Using TensorFlow backend.


In [2]:
nlp = spacy.load("en_vectors_web_lg")

# Creating vectors if they don't exist

In [10]:
text_df = pd.read_csv("data/moral_samples.csv")
text_df.head()

,sample,moral_saliency,moral_intrinsicality,notes
0,All human beings have inherent worth.,1,3,NaN
1,It's wrong to exploit people you have power over.,1,2,NaN
2,Jeff Bezos is an evail man.,1,1,NaN
3,All human beings have a brain that resides in ...,0,0,NaN
4,It's wrong to assume that one plus one is equa...,0,0,NaN


vectors_df = text_df.rename(index=str, columns={"smaple": "text"})
vectors_df["text"] = vectors_df["sample"]
vectors_df = vectors_df.drop(["sample", "notes"], axis=1)
vectors_df["vector"] = vectors_df["text"].apply(lambda text: nlp(text).vector)
vectors_df.head()

In [12]:
vectors_df.to_hdf("data/moral_vectors.hdf", key="df")

/Users/wlifferth/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:2378: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['text', 'vector']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [139]:
vectors_df = pd.read_hdf("data/moral_vectors.hdf", key="df")

# Decide what our Test/Train split

### Run one of these

In [140]:
# Random split
msk = np.random.rand(len(vectors_df)) < 0.2
test_df = vectors_df[msk]
train_df = vectors_df[~msk]

In [231]:
# Make Sure our positive test cases are abstract
positive_df = vectors_df[vectors_df["moral_saliency"] == 1]
negative_df = vectors_df[vectors_df["moral_saliency"] == 0]
negative_train = negative_df.sample(frac=0.5)
negative_test = negative_df.drop(negative_train.index)
train_df = pd.concat([positive_df[positive_df["moral_intrinsicality"] == 3], negative_train]).sample(frac=1)
test_df = pd.concat([positive_df[positive_df["moral_intrinsicality"] != 3], negative_test]).sample(frac=1)

In [208]:
# Make Sure our positive test cases are abstract
positive_df = vectors_df[vectors_df["moral_saliency"] == 1]
negative_df = vectors_df[vectors_df["moral_saliency"] == 0]
msk = np.random.rand(len(negative_df)) < 0.5
train_df = pd.concat([positive_df[positive_df["moral_intrinsicality"] == 2], negative_df[msk]]).sample(frac=1)
test_df = pd.concat([positive_df[positive_df["moral_intrinsicality"] != 2], negative_df[~msk]]).sample(frac=1)

In [ ]:
# Make Sure our positive test cases are abstract
positive_df = vectors_df[vectors_df["moral_saliency"] == 1]
negative_df = vectors_df[vectors_df["moral_saliency"] == 0]
msk = np.random.rand(len(negative_df)) < 0.5
train_df = pd.concat([positive_df[positive_df["moral_intrinsicality"] == 1], negative_df[msk]]).sample(frac=1)
test_df = pd.concat([positive_df[positive_df["moral_intrinsicality"] != 1], negative_df[~msk]]).sample(frac=1)

# Generate Some "Synthetic" Data

In [232]:
noisy_df = pd.DataFrame({"moral_saliency": [], "moral_intrinsicality": [], "text": [], "vector": []})

### Inset Random Char

In [233]:
rand_char_insert_data = {"moral_saliency": [], "moral_intrinsicality": [], "text": [], "vector": []}

In [234]:
chars = list("abcdefghijklmnopqrstuvwxyz0123456789")
for index, row in train_df.iterrows():
    text = row["text"]
    insert_index = randint(0, len(text))
    rand_char = choice(chars)
    new_text = text[:insert_index] + rand_char + text[insert_index:]
    rand_char_insert_data["text"].append(new_text)
    rand_char_insert_data["moral_saliency"].append(row["moral_saliency"])
    rand_char_insert_data["moral_intrinsicality"].append(row["moral_intrinsicality"])
    rand_char_insert_data["vector"].append(nlp(new_text).vector)

In [235]:
noisy_df = pd.concat([noisy_df, pd.DataFrame(rand_char_insert_data)])
print(len(noisy_df))
noisy_df.head()

40


,moral_saliency,moral_intrinsicality,text,vector
0,0.0,0.0,I think this is thes first book I've ever hated.,"[-0.039986417, 0.20880084, -0.17606848, -0.274..."
1,1.0,3.0,Drug addiction can rnob a person of their free...,"[-0.094240494, 0.18061209, -0.11198693, -0.013..."
2,1.0,3.0,A woman has a right to herj own body and to ma...,"[-0.0030626676, 0.17343123, -0.16255562, 0.155..."
3,0.0,0.0,"These highlighters come in pretty colors, but ...","[-0.06329271, 0.17525336, -0.14811607, -0.1231..."
4,1.0,3.0,All human beings have inhere9nt worth.,"[-0.21430875, -0.029681427, -0.1463257, -0.206..."


### Delete Random Char

In [236]:
rand_char_del_data = {"moral_saliency": [], "moral_intrinsicality": [], "text": [], "vector": []}

In [237]:
for index, row in train_df.iterrows():
    text = row["text"]
    delete_index = randint(0, len(text))
    rand_char = choice(chars)
    new_text = text[:delete_index-1] + text[delete_index:]
    rand_char_del_data["text"].append(new_text)
    rand_char_del_data["moral_saliency"].append(row["moral_saliency"])
    rand_char_del_data["moral_intrinsicality"].append(row["moral_intrinsicality"])
    rand_char_del_data["vector"].append(nlp(new_text).vector)

In [238]:
noisy_df = pd.concat([noisy_df, pd.DataFrame(rand_char_del_data)])
print(len(noisy_df))
noisy_df.head()

80


,moral_saliency,moral_intrinsicality,text,vector
0,0.0,0.0,I think this is thes first book I've ever hated.,"[-0.039986417, 0.20880084, -0.17606848, -0.274..."
1,1.0,3.0,Drug addiction can rnob a person of their free...,"[-0.094240494, 0.18061209, -0.11198693, -0.013..."
2,1.0,3.0,A woman has a right to herj own body and to ma...,"[-0.0030626676, 0.17343123, -0.16255562, 0.155..."
3,0.0,0.0,"These highlighters come in pretty colors, but ...","[-0.06329271, 0.17525336, -0.14811607, -0.1231..."
4,1.0,3.0,All human beings have inhere9nt worth.,"[-0.21430875, -0.029681427, -0.1463257, -0.206..."


### Substitue Random Char

In [239]:
rand_char_sub_data = {"moral_saliency": [], "moral_intrinsicality": [], "text": [], "vector": []}

In [240]:
for index, row in train_df.iterrows():
    text = row["text"]
    sub_index = randint(0, len(text))
    rand_char = choice(chars)
    new_text = text[:sub_index-1] + rand_char + text[sub_index:]
    rand_char_sub_data["text"].append(new_text)
    rand_char_sub_data["moral_saliency"].append(row["moral_saliency"])
    rand_char_sub_data["moral_intrinsicality"].append(row["moral_intrinsicality"])
    rand_char_sub_data["vector"].append(nlp(new_text).vector)

In [241]:
noisy_df = pd.concat([noisy_df, pd.DataFrame(rand_char_sub_data)])
print(len(noisy_df))
noisy_df.head()

120


,moral_saliency,moral_intrinsicality,text,vector
0,0.0,0.0,I think this is thes first book I've ever hated.,"[-0.039986417, 0.20880084, -0.17606848, -0.274..."
1,1.0,3.0,Drug addiction can rnob a person of their free...,"[-0.094240494, 0.18061209, -0.11198693, -0.013..."
2,1.0,3.0,A woman has a right to herj own body and to ma...,"[-0.0030626676, 0.17343123, -0.16255562, 0.155..."
3,0.0,0.0,"These highlighters come in pretty colors, but ...","[-0.06329271, 0.17525336, -0.14811607, -0.1231..."
4,1.0,3.0,All human beings have inhere9nt worth.,"[-0.21430875, -0.029681427, -0.1463257, -0.206..."


### Delete Random Word

In [242]:
rand_word_del_data = {"moral_saliency": [], "moral_intrinsicality": [], "text": [], "vector": []}

In [243]:
for index, row in train_df.iterrows():
    words = row["text"].split()
    del_index = randint(0, len(text))
    new_text = " ".join(words[:sub_index-1] + words[sub_index:])
    rand_word_del_data["text"].append(new_text)
    rand_word_del_data["moral_saliency"].append(row["moral_saliency"])
    rand_word_del_data["moral_intrinsicality"].append(row["moral_intrinsicality"])
    rand_word_del_data["vector"].append(nlp(new_text).vector)

In [244]:
noisy_df = pd.concat([pd.DataFrame(rand_word_del_data), noisy_df])
print(len(noisy_df))
noisy_df.head()

160


,moral_saliency,moral_intrinsicality,text,vector
0,0.0,0.0,I think this is the first book I've ever hated.,"[-0.012673001, 0.23964083, -0.18042934, -0.229..."
1,1.0,3.0,Drug addiction can rob a person of their freed...,"[-0.1253105, 0.18894875, -0.13488442, -0.01481..."
2,1.0,3.0,A woman has a right to her own body and to dec...,"[0.019030118, 0.18027502, -0.1611136, 0.199858..."
3,0.0,0.0,"These highlighters come in pretty colors, but ...","[-0.080520436, 0.1890338, -0.14695458, -0.1265..."
4,1.0,3.0,All human beings have inherent worth.,"[-0.24293303, 0.035708573, -0.10840428, -0.148..."


# Add Synthetic Data In

In [245]:
train_df = pd.concat([train_df, noisy_df])
len(train_df)

200

# Formatting data for training

In [246]:
input_train_data = np.stack(train_df.sample(frac=1)['vector'].values, axis=0)
input_train_data.shape

(200, 300)

In [247]:
input_test_data = np.stack(test_df.sample(frac=1)['vector'].values, axis=0)
input_test_data.shape

(60, 300)

In [248]:
label_train_data = train_df[['moral_saliency']].values
label_train_data.shape

(200, 1)

In [249]:
label_test_data = test_df[['moral_saliency']].values
label_test_data.shape

(60, 1)

# Training a Model

In [255]:
model = keras.Sequential()
model.add(keras.layers.Dense(300, activation=tf.nn.relu))
model.add(keras.layers.Dropout(0.2, input_shape=(300,)))
model.add(keras.layers.Dense(8, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

In [256]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [257]:
history = model.fit(input_train_data,
                    label_train_data,
                    epochs=16,
                    batch_size=64,
                    validation_data=(input_test_data, label_test_data))

Train on 200 samples, validate on 60 samples
Epoch 1/16
200/200 [==============================] - 1s 7ms/step - loss: 0.6832 - acc: 0.6000 - val_loss: 0.7467 - val_acc: 0.4167
Epoch 2/16
200/200 [==============================] - 0s 148us/step - loss: 0.6594 - acc: 0.6250 - val_loss: 0.7993 - val_acc: 0.4167
Epoch 3/16
200/200 [==============================] - 0s 186us/step - loss: 0.6506 - acc: 0.6250 - val_loss: 0.7863 - val_acc: 0.4167
Epoch 4/16
200/200 [==============================] - 0s 147us/step - loss: 0.6401 - acc: 0.6250 - val_loss: 0.7787 - val_acc: 0.4167
Epoch 5/16
200/200 [==============================] - 0s 131us/step - loss: 0.6306 - acc: 0.6400 - val_loss: 0.7781 - val_acc: 0.4167
Epoch 6/16
200/200 [==============================] - 0s 120us/step - loss: 0.6212 - acc: 0.6400 - val_loss: 0.7493 - val_acc: 0.4167
Epoch 7/16
200/200 [==============================] - 0s 124us/step - loss: 0.6162 - acc: 0.6400 - val_loss: 0.7375 - val_acc: 0.4333
Epoch 8/16
200/200 

In [162]:
for index, row in vectors_df.iterrows():
    prediction = model.predict(np.array([row["vector"]]))
    if np.round(model.predict(np.array([row["vector"]]))[0][0]) != row["moral_saliency"]:
        print(row["text"], end="\n\n")
        print("moral_saliency: ", row["moral_saliency"], "prediction: ", prediction[0][0], end="\n\n\n")

All human beings have inherent worth.

moral_saliency:  1 prediction:  0.3063171


Jeff Bezos is the CEO of Amazon and Blue Origin (a company that makes rockets).

moral_saliency:  0 prediction:  0.566317


Abortion is wrong because ending the life of a fetus is murder.

moral_saliency:  1 prediction:  0.47532338


A woman has a right to her own body and to make decisions about her reproducitve health.

moral_saliency:  1 prediction:  0.46180832


In a clinical sense, abortion is the deliberate termination of a human pregnancy.

moral_saliency:  0 prediction:  0.575553


Many states have rules requiring clinics to have admitting privledges at a hospital in order to perform abortions.

moral_saliency:  0 prediction:  0.57993996


There is a lot of contravesry around abortion access, and Planned Parenthood sits at the middle of it.

moral_saliency:  0 prediction:  0.65074307


Members of a nation have a fundamental duty to support that thing of which they're a part.

moral_saliency:  1 p